# Predicting water baseflow

## Load and clean the database

In [87]:
# imports
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('RRCA_baseflow.csv')

# Remove whitespace in col names
df.columns = df.columns.str.strip()

# create a modified date column to deal with idiosyncrasies
df['NewDate'] = df['Date'] - 693963

# create a positive Irrigation_pumping column to visualize the data better
df['PositiveIrrigation'] = df['Irrigation_pumping'] * -1

## Initial scatterplots to visualize the data

For each river segment, what is the relationship between observed flow and precipitation, irrigation, and evapotranspiration:

In [ ]:
# Get unique segment IDs
unique_segment_ids = df['Segment_id'].unique()

# Sort the unique segment IDs in ascending order
unique_segment_ids_sorted = sorted(unique_segment_ids)

# Create a figure with subplots for each unique segment ID
fig, axs = plt.subplots(len(unique_segment_ids_sorted), 3, figsize=(15, 5 * len(unique_segment_ids_sorted)))

# Loop through each unique segment ID and plot the scatter plots
for i, segment_id in enumerate(unique_segment_ids_sorted):
    df_filtered = df[df['Segment_id'] == segment_id]
    # df_filtered.plot(kind='scatter', x='Irrigation_pumping', y='Observed', ax=axs[i, 0], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='PositiveIrrigation', y='Observed', ax=axs[i, 0], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='Precipitation', y='Observed', ax=axs[i, 1], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='Evapotranspiration', y='Observed', ax=axs[i, 2], title=f'Segment ID: {segment_id}')

plt.tight_layout()
plt.show()

# Note: I tried flipping the axes here so that the zeros are on the x-axis, but it looked messier to me

### Is there a relationship between observed waterflow and evapotranspiration, precipitation, or irrigaion? 

Our tentative observations based on these scatterplots:
- As there is more evapotranspiration in an area, the observed waterflow is lower
- As more water is being pumped out for irrigation, the observed waterflow is lower
- As there is more precipation in an area, the observed waterflow is higher

## How does the observed flow change over time?

In [ ]:
# Get unique segment IDs
unique_segment_ids = df['Segment_id'].unique()

# Sort the unique segment IDs in ascending order
unique_segment_ids = sorted(unique_segment_ids)

# For each segment, plot the observed baseflow over time
fig, axs = plt.subplots(len(unique_segment_ids), 1, figsize=(15, 5 * len(unique_segment_ids)))

# Loop through each unique segment ID and plot the scatter plots
for i, segment_id in enumerate(unique_segment_ids):
    df_filtered = df[df['Segment_id'] == segment_id]
    df_filtered.plot(kind='scatter', x='NewDate', y='Observed', ax=axs[i], title=f'Segment ID: {segment_id}')

plt.tight_layout()
plt.show()


## How do the variables we're tracking change over time?

In [ ]:
# Get unique segment IDs
unique_segment_ids = df['Segment_id'].unique()

# Sort the unique segment IDs in ascending order
unique_segment_ids = sorted(unique_segment_ids)

# For each segment, plot PositiveIrrigation, Precipitation, and Evapotranspiration over time
fig, axs = plt.subplots(len(unique_segment_ids), 3, figsize=(15, 5 * len(unique_segment_ids)))

# Loop through each unique segment ID and plot the scatter plots
for i, segment_id in enumerate(unique_segment_ids):
    df_filtered = df[df['Segment_id'] == segment_id]
    df_filtered.plot(kind='scatter', x='NewDate', y='PositiveIrrigation', ax=axs[i, 0], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='NewDate', y='Precipitation', ax=axs[i, 1], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='NewDate', y='Evapotranspiration', ax=axs[i, 2], title=f'Segment ID: {segment_id}')

plt.tight_layout()
plt.show()

## Estimating ("Learning") Model Coefficients

In [ ]:
# create X and y
feature_cols = ['PositiveIrrigation', 'Precipitation', 'Evapotranspiration']
# feature_cols = ['PositiveIrrigation']
# feature_cols = ['Precipitation']
# feature_cols = ['Evapotranspiration']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

## Interpreting Model Coefficients

The baseline value of the observed flow, with these three variables being zero, is 3.2398

A "unit" increase in irrigation pumping is __associated with__ a -18.73476 "unit" change in observed flow. As the amount of water being pumped _increases_, the flow _decreases_.

A "unit" increase in evapotranspiration is __associated with__ a -0.3128 "unit" change in observed flow. As the amount of water evapotranspiring _increases_, the flow _decreases_.

A "unit" increase in precipitation is __associated with__ a 1.56127 "unit" increase in observed flow. As the amount of precipitation _increases_, the flow _increases_.

In [ ]:
feature_cols = ['PositiveIrrigation', 'Evapotranspiration']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

The baseline value of the observed flow, if only irrigation and evapotranspiration are zero, and precipitation isn't zero, is _considerably_ higher.

In [ ]:
feature_cols = ['PositiveIrrigation', 'Precipitation', 'Evapotranspiration']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

Something fascinating here is that this is saying that the baseline value of the observed flow will be even higher if only irrigation is zero, and evapotranspiration and precipitation are not zero. It seems that irrigation is the greatest decreaser in the baseflow of the river.

### How strong are the relationships between observed waterflow and evapotranspiration, precipitaiton, or irrigation?

## Plotting the Least Squares Line

Let's make predictions for the __smallest and largest observed values of each variable__, and then use the predicted values to plot the least squares line:

In [ ]:
# create a DataFrame with the minimum and maximum values of PositiveIrrigation, Precipitation, and Evapotranspiration
X_new = pd.DataFrame({'PositiveIrrigation': [df.PositiveIrrigation.min(), df.PositiveIrrigation.max()], 'Precipitation': [df.Precipitation.min(), df.Precipitation.max()], 'Evapotranspiration': [df.Evapotranspiration.min(), df.Evapotranspiration.max()]})
# X_new = pd.DataFrame({'Irrigation_pumping': [df.Irrigation_pumping.min(), df.Irrigation_pumping.max()]})
X_new

The first row above is the minimum values of these features.
The second row is the maximum values of these features.

In [ ]:
# make predictions for those x values and store them
preds = lm.predict(X_new)
preds

3.23984852 is the predicted value of Observed (the observed baseflow) for the minimum values. 3.2089427 is the predicted value of Observed for the maximum values.

## Confidence in our Model

We can plot the confidence intervals using Seaborn. In Seaborn, the data is binned according to the independent variable and then a band containing 95% of the means of the bin is shown.

In [ ]:
import seaborn as sns

# Plot the sampled data with the confidence interval
plt.figure()
ax = sns.regplot(x='PositiveIrrigation', y='Observed', data=df, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Evapotranspiration', y='Observed', data=df, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Precipitation', y='Observed', data=df, ci=95, scatter_kws={'s': 1})

The shaded region shows all possible regression curves within the 95% confidence interval.

## Hypothesis Testing and p-values

As it relates to model coefficients, here is the conventional hypothesis test:
- **null hypothesis:** There is no relationship between irrigation pumping and observed flow (and thus $\beta_1$ equals zero)
- **alternative hypothesis:** There is a relationship between irrigation pumping and observed flow (and thus $\beta_1$ is not equal to zero)

We reject the null (and thus believe the alternative) if the 95% confidence interval __does not include zero__. The p-value represents the probability that the coefficient is actually zero:

In [ ]:
# use statsmodel to compute the confidence interval
# this is the standard import if you're using "formula notation"
import statsmodels.formula.api as smf

# lm = smf.ols(formula='Observed ~ Irrigation_pumping + Precipitation + Evapotranspiration', data=df).fit()
print("============ PositiveIrrigation ============")
lm = smf.ols(formula='Observed ~ PositiveIrrigation', data=df).fit()
display(lm.pvalues) # print the p-values for the model coefficients
display(lm.params) # print the coefficients
display(lm.conf_int()) # print the confidence intervals for the model coefficients
display(lm.rsquared) # print the R-squared value for the model

print("============ Evapotranspiration ============")
lm = smf.ols(formula='Observed ~ Evapotranspiration', data=df).fit()
display(lm.pvalues)
display(lm.params)
display(lm.conf_int())
display(lm.rsquared)

print("============ Precipitation ============")
lm = smf.ols(formula='Observed ~ Precipitation', data=df).fit()
display(lm.pvalues)
display(lm.params)
display(lm.conf_int())
display(lm.rsquared)

(With a 95% confidence interval for all attributes of the following variables)

(Values are truncated at the whole number in this summary)

We see that, with a 95% confidence interval, the true baseflow of the river is between 24 and 25 without any water being taken for irrigation. With a 95% confidence interval, each unit of irrigation water being pumped from the river results in 8 to 14 _less_ units of true baseflow for the river. The p-value for `PositiveIrrigation` is low, suggesting that it is very unlikely that this observed relationship between irrigation and baseflow is just a coincidence (and not statistically significant).

Additionally, we see that the true baseflow of the river is between 19 and 22 without any evapotranspiration. Each unit of evapotranspiration on the river results in 0.8 to 1.4 _more_ units of true baseflow for the river. The p-value for `Evapotranspiration` is also low, suggesting that it is very unlikely that this observed relationship is just a coincidence.

Finally, we see that the true baseflow of the river (again, with a 95% confidence interval, as with the other variables) is between 0.95 and 3.9 without any precipitation. Each unit of precipitation results in 1.36 to 1.536 _more_ units of true baseflow for the river. The p-value for `Precipitation` is _extremely_ low, indicating that it is extremely unlikely that this observed relationship is just a coincidence.

The R-squared values are pretty low for all of these variables, which indicates that none of these models (of just one variable fit to the data across the whole river) is a very good fit for our observed baseflow data. This is probably because there is so much variance in observed flow throughout the year, as well as across different sections of the river.

## How Well Does the Model Fit the data? (R-squared)

### (a refresher on what R-squared means in our results)

The most common way to evaluate the overall fit of a linear model is by the **R-squared** value. R-squared is the **proportion of variance explained**, meaning the proportion of variance in the observed data that is explained by the model, or the reduction in error over the **null model**. (The null model just predicts the mean of the observed response, and thus it has an intercept and no slope.)

$R^2 = 1 - \frac{\text{residual}}{\text{total variation}} = \frac{\sum_{i=1}^N (y_i-\hat{y}_i)^2}{\sum_{i=1}^N (y_i-\overline{y})^2}$

where $\overline{y}$ is the average value and $\hat{y}_i$ is the predicted value.

R-squared is between 0 and 1, and higher is better because it means that more variance is explained by the model. This is a way of normalizing the raw residual.

What is a "good" R-squared value? It's hard to say. The threshold for a good R-squared value depends widely on the domain. Therefore, it's most useful as a tool for comparing different models.

## Predict the observed flow of one segment of the river

The scatterplot for Segment 176 looked to have a stronger relationship between observed baseflow and evapotranspiration and precipitation. 

If we look at the observed baseflow over time, we can see that the baseflow is much higher before NewDate 18500, and is much lower after NewDate 18500. Therefore, to more accurately predict the observed baseflow for segment 176 of the river, we must determine what time we're predicting the observed baseflow for. We'll pick dates before 18500.

Now, look at the regression lines for each variable in that segment. 

Then, model just segment 176, fitting the data for the relationship between `Observed` and whichever variable has the strongest regression line:

In [ ]:
import seaborn as sns

# create a table that is just for Segment_id = 170
df_176 = df[df['Segment_id'] == 176]

# Filter df_176 to include only rows where NewDate is less than 18500
df_176_lower_date = df_176[df_176['NewDate'] < 18500]

# Filter df_176 to include only rows where NewDate is greater than 18500
# df_176_higher_date = df_176[df_176['NewDate'] >= 18500]

# Plot the sampled data with the confidence interval
plt.figure()
ax = sns.regplot(x='PositiveIrrigation', y='Observed', data=df_176_lower_date, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Evapotranspiration', y='Observed', data=df_176_lower_date, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Precipitation', y='Observed', data=df_176_lower_date, ci=95, scatter_kws={'s': 1})

Here we can see that `Evapotranspiration` and `Precipitation` have the strongest relationship with `Observed`, so we will use them to fit our model.

In [ ]:
print("============ Evapotranspiration, Segment 170 ============")
lm = smf.ols(formula='Observed ~ Precipitation + Evapotranspiration', data=df_176_lower_date).fit()
display(lm.pvalues) # print the p-values for the model coefficients
display(lm.params) # print the coefficients
display(lm.conf_int()) # print the confidence intervals for the model coefficients
display(lm.rsquared) # print the R-squared value for the model

Using just `PositiveIrrigation`, we got an R-value of basically zero. That's not surprising, as our scatterplots indicate that there was no irrigation happening on this segment of the river before NewDate 18500.

Using just `Precipitation`, we got an R-value of 0.04573918848025471. This is pretty low.

Using just `Evapotranspiration`, we got an R-value of 0.4852108044654765. This is significantly better than the R-value we got for `Precipitation`.

Using `Precipitation` and `Evapotranspiration` _together_, we got an R-value of 0.5148607593626253. This is even better than just using `Evapotranspiration`.

Our R-value is also considerably higher here than it was using `Evapotranspiration` and `Precipitation` on the whole river, over all date ranges (the R-value for that was 0.07641857115830208).

Here we see that, if we want to predict a value for the observed baseflow of the river, our prediction is much more accurate if we can narrow down what segment of the river we want the baseflow for, and what date range we want to predict for. Then we can determine the best variables to use in our model. After all of that is done, our model will be much more accurate than a model based on the whole river generally.

From this model, we see that when `Evapotranspiration` and `Precipitation` are zero, the true observed baseflow is (with 95% accuracy) between 23 and 44.

## Using the Model for Prediction

If we want to predict the baseflow of this segment of the river, using the model we created above, we can use `Statsmodels` to make a prediction using values we give it. In this example, we predict no irrigation and a fair amount of precipitation and evapotranspiration:

In [ ]:
# you have to create a DataFrame since the Statsmodels formula interface expects it
X_new = pd.DataFrame({'PositiveIrrigation': [0], 'Precipitation': [25], 'Evapotranspiration': [1.2]})
display(X_new)

In [ ]:
# use the model to make predictions on a new value
lm.predict(X_new)

Here, we get a prediction of 38.563747 for `Observed`, which matches values that we have in the database pretty well. 

These findings could and _should_ be generalized to the rest of the river. If we want to predict the value of `Observed`, it is better to predict that value for a specific _section_ of the river, at a specific _time_, and then to go off of the variable with the strongest relationship to `Observed` in that section of the river, at that time.